(с) В.И. Фирсанова | Основы программирования на Python, НИУ ВШЭ

Используйте этот шаблон для разработки итогого проекта по курсу "Основы программирования на Python" 2023

Курсивом выделены образцы заполнения шаблона. Эти элементы следует заменить своими данными. Также везде следует заменить код на собственный, однако не воспрещается пользоваться предложенным кодом в качестве референса


# Название проекта: *Анализ тональности в социальных сетях*


## I. Введение
   - Цели и задачи проекта
   - Обзор проекта

Цель проекта: *произвести анализ тональности данных из социальной сети ВКонтакте*

Задачи:

1. *Найти датасет для анализа тональности текста на платформе Kaggle*
2. *Произвести предварительную обработку данных*
3. *Визуализировать распределение тональности в датасете*
4. *Выявить наиболее частотные коллокации*
5. *Использовать алгоритм TF-IDF для создания векторных представлений текста*
6. *Применить готовую модель для анализа тональности из библиотеки TextBLob*
7. *Визуализировать результат обработки на матрице ошибок*
8. *Вывести несколько правильных и ошибочных результатов, чтобы вявить слабые и сильные стороны выбранного алгоритма*

*Результаты проекта могут использоваться в качестве основы для разработки модели машинного обучения для анализа тональности текста.*

## II. Датасет
   - Выбрать, описать и загрузить источник используемых данных

*Датасет выгружен по ссылке [example.com](https://). Данные собирались с помощью ВКонакте API для Python.*


In [ ]:
import pandas as pd

data_path = "/content/sentiment.csv"
df = pd.read_csv(data_path)

df.head()

FileNotFoundError: ignored

Какие данные описаны в датасете?

Здесь можно кратко описать, из каких столбцов состоят ваши данные, какая используется разметка и т.д.

In [ ]:
df.info()

Какие типы данных хранятся в этом датасете?
Какой объем памяти занимают данные?
Какие выводы можно сделать на основе этих фактов?

In [ ]:
df.describe()

Опишите свои данные, назовите средние значения, минимумы и максимумы, общее количество объектов в датасете. Сфокусируйтесь на тех данные, которые могут пригодиться вам для решения вашей задачи.

   - Чистка данных (выбираете то, что подходит для ваших данных, можно использовать как нативные способы, так и библиотеки):
     - Удаление специальных символов, URL, ненужных столбцов
     - Токенизация
     - Лемматизация
     - Стемминг
     - Удаление стоп-слов

In [ ]:
# Удаление пустых ячеек

df.dropna(inplace=True)

In [ ]:
# Удаление столбцов, которые не требуются вам для решения задачи

df.drop(['3364'], axis=1, inplace=True)

df.head()

In [ ]:
# Изменить названия столбцов

df.rename(columns={'Facebook': 'source', 'Irrelevant': 'label', 'I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣': 'text'}, inplace=True)
df.head()

In [ ]:
# Изменить типы данных

df['label'] = df['label'].astype('category')
df['label']

In [ ]:
# Привести данные к нижнему регистру
# Очистить данные от лищних символов с помощью регулярных выражений

df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace('[^\w\s]', '', regex=True)
df['text']

   - Разметка (выбираете то, что подходит для ваших данных, можно использовать как нативные способы, так и библиотеки):
     - Частеречная разметка
     - Автоматическая разметка, необходимая для решения вашей задачи
     - Ручная разметка, необходимая для решения вашей задачи

In [ ]:
# частеречная разметка с NLTK

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# токенизация

df['Tokens'] = df['text'].apply(nltk.word_tokenize)

# частереная разметка

df['POS_Tags'] = df['Tokens'].apply(nltk.pos_tag)

# извлечение POS-тегов

df['POS'] = df['POS_Tags'].apply(lambda tags: [pos for word, pos in tags])

df.head()


## III. Предварительный анализ данных
   - Визуализируйте данные с помощью таких графиков, как:
     - коррелограмма,
     - гистограмма,
     - точечная диаграмма,
     - ящик с усами

In [ ]:
import matplotlib.pyplot as plt

# гистограмма

plt.hist(df['label'], bins=20, color='skyblue', edgecolor='black')

# подписать оси x, y и название гистограммы
plt.xlabel('Количество текстов данного класса')
plt.ylabel('Сколько раз встречается в датасете')
plt.title('Распределение классов в датасете')

# отобразить
plt.show()

In [ ]:
# точечная диаграмма для датасета об автомобиля строится, чтобы определить наличие связи
# между стоимостью автомобиля и количеством лошадиных сил

plt.scatter(df['price'], df['horsepower'], alpha=0.5, color='green')

In [ ]:
import seaborn as sns

# Выбросы - это данные, которые сильно отклоняются (гораздо больше или меньше) минимальных или максимальных величин
# Например, у нас в данных самые большие значения равны 90-100, но есть одно или два значения равные 250
# Значения, равные 250, мы назовем выбросами в данных
# Они могут негативно влиять на статистику, например, искажать реальную картинку среднего значения
# Чтобы их найти, можно использовать график "ящик с усами"
# Этот график так назван за внешнее сходство с ящиком, у которого есть антенны

# Создать ящик с усами для столбца со стоимостью автомобиля
sns.boxplot(x=df['price'], color='purple')

In [ ]:
# Вычислить корреляцию - связи между всеми элементами датасета

correlation_matrix = df.corr()

# Визуализировать связи между всеми элементами датасета
# Чем теплее оттенок, тем сильнее связь, и наоборот

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

Сделайте выводы по каждой выбранной визуализации:
- сбалансированная выборка или нет,
- есть связь или зависимость между выбранными данными или нет,
- возрастает или убывает сила связи,
- есть ли выбросы в данных...

   - Выведите наиболее частотные коллокации и слова (возможные методы):
     - создайте частотные словари,
     - визуализация с помощью облака слов

In [ ]:
# пример частотного словаря для одного текста в датасете

data = pd.Series(df['Tokens'][50])

frequency_dict = data.value_counts().to_dict()

frequency_dict

In [ ]:
from wordcloud import WordCloud

# облако слов для одного текста

wordcloud = WordCloud().generate(df['text'][50])

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Какие выводы позволяет сделать частотный словарь? Какова практическая значимость такой работы: можно ли создать с ее помощью векторное представление текста, выявить стоп-слова? Что еще можно сделать с частотным словарем и облаком слов?

## IV. Векторное представление
   - Построить любое векторное представление (ориентируйтесь на свой опыт в Python):
     - Bag-of-Words
     - TF-IDF
     - Word2Vec
   - Если с построением векторного представления возникают трудности, выделите такие характеристики текстов:
     - минимальная, максимальная, средняя длина текста в датасете...

In [ ]:
# образец двухмерного (для двух предложений) мешка слов

df1 = pd.DataFrame(glossaries[0], columns=['word', 'sentence 0'])
df2 = pd.DataFrame(glossaries[2], columns=['word', 'sentence 2'])

df = df1.merge(df2, on='word')

df.columns = ['word', 'sentence 0', 'sentence 1']

df


## V. Модель NLP
   - Выбрать и применить к данным готовую модель NLP для решения какой-либо задачи:
     - анализ тональности,
     - классификация,
     - суммаризация,
     - разметка именованных сущностей,
     - разметка синтаксиса или частей речи...

In [ ]:
# применяем TextBlob для анализа тональности к данным

from textblob import TextBlob

for index, row in df.iterrows():
    text = row['text']
    sentiment = TextBlob(text)

    df.loc[index, 'Polarity'] = sentiment.sentiment.polarity
    df.loc[index, 'Subjectivity'] = sentiment.sentiment.subjectivity

- Оценка результата:
     - данные можно визуализировать (например, с помощью матрицы ошибок) или оценить метриками (например, полнота, точность и F-мера)
     - вы должны обосновать свой выбор визуализации и метрик оценки

In [ ]:
# визуализация результатов работы модели

plt.figure(figsize=(10, 6))

plt.hist(df['Polarity'], bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Polarity Score')
plt.ylabel('Frequency')
plt.title('Distribution of Polarity Scores')

plt.figure(figsize=(10, 6))

plt.hist(df['Subjectivity'], bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Subjectivity Score')
plt.ylabel('Frequency')
plt.title('Distribution of Subjectivity Scores')

plt.show()

## VI. Выводы
   - Представьте свои рассуждения о проделанной работе:
     - с какими трудностями вы столкнулись,
     - что нового вы узнали в нашем курсе и в процессе работы над проектом,
     - какие выводы сделали для себя, что открыли, что заберете с собой для будущих работ
   - Перспективы:
     - поделитесь идеями о том, как можно развить ваш проект

## VII. Презентация
   - Будьте готовы к устной презентации:
     - разработайте короткую презентацию (5 слайдов), в которой вы опишете цели, задачи, процесс работы и результаты
   - Иллюстрации:
     - представьте основные выводы, репрезентативные визуализации

## VIII. Загрузка
   - Загрузить код и презентацию в свой репозиторий